# The Palatable Diet Problem  

**Problem Description.** The Diet Problem is the first large-scale optimization problem to be solved with the Simplex algorithm by Jack Laderman in [1947](https://www.mpi-inf.mpg.de/fileadmin/inf/d1/teaching/winter18/Ideen/Materialien/Dantzig-Diet.pdf). The basic formulation of this problem consists of minimizing the cost of a food basket while meeting the specified nutrient requirements. In this notebook, we solve The Palatable Diet Problem (TPDP), where the basic model is extended with a constraint on the food basket palatability. An explicit formula of the palatability constraint is unknown, but we have data on several food baskets and the respective palatability score. First, we define a conceptual model with the *known constraint*. Then, OptiCL is used to learn and embed the palatability constraint.  
(*TPDP is part of a larger optimization problem which simultaneously  optimizes  the  food  basket  to  be  delivered,  the  sourcing  plan,  the  delivery  plan, and  the  transfer  modality  of  a  month-long  food  supply in a Wolrd Food Program setting ([Maragno et al., 2021])*).

<font color='#808080'>**Objective function:** minimize the total cost of the food basket.</font>  
$\min_{\boldsymbol{x}} c^\top \boldsymbol{x}$

*subject to* 

<font color='#808080'>**Nutritional constraints:** for each nutrient $l\in\mathcal{L}$, at least meet the minimum required level.</font>  
$ \sum_{k \in \mathcal{K}} nutval_{kl} x_{k} \geq nutreq_{l}, \ \ \ \forall l\in\mathcal{L},$   
<font color='#808080'>**Palatability constraints:** the food basket palatability has to be at least equal to $t$.</font>  
$ y \geq t,$  
<font color='#808080'>**Learned predictive model:** the palatability is defined using a predictive model.</font>  
$ y = \hat{h}(\boldsymbol{x}),$   
<font color='#808080'>**Non negativity constraints.**</font>  
$ x_{k} \geq 0, \ \ \ \forall k \in \mathcal{K}.$  

In [1]:
import pandas as pd
import numpy as np
import time
import sys
import os
sys.path.append(os.path.abspath('../../src'))  # TODO: has to be changed
import opticl
from pyomo import environ
from pyomo.environ import *
import embed_mip as em
from imp import reload
import matplotlib.pyplot as plt

### Data Loading  
**nutr_val**: nutritional values for each of the 25 foods  
**nutr_req**: 11 nutrition requirements  
**cost_p**: vector of procurement costs  
**dataset**: dataframe of food basket instances and relative palatability score

In [2]:
nutr_val = pd.read_excel('../WFP/processed-data/Syria_instance.xlsx', sheet_name='nutr_val', index_col='Food')
nutr_req = pd.read_excel('../WFP/processed-data/Syria_instance.xlsx', sheet_name='nutr_req', index_col='Type')
cost_p = pd.read_excel('../WFP/processed-data/Syria_instance.xlsx', sheet_name='FoodCost', index_col='Supplier').iloc[0,:]
dataset = pd.read_csv('../WFP/processed-data/WFP_palatability_robustness.csv').sample(frac=1)

nutr_val.drop(['Salt','Sugar'], axis=0, inplace = True)
nutr_val.drop(['Iodine(ug)'], axis=1, inplace = True)
nutr_req.drop(['Iodine(ug)'], axis=1, inplace = True)
dataset.drop(['Salt','Sugar'], axis=1, inplace=True)
dataset = dataset.round(4)
dataset.to_csv('../WFP/processed-data/WFP_dataset_processed.csv', index=False)
dataset

,Beans,Bulgur,Cheese,Fish,Meat,CSB,Dates,DSM,Milk,Lentils,...,Sorghum,Soya-fortified bulgur wheat,Soya-fortified maize meal,Soya-fortified sorghum grits,Soya-fortified wheat flour,Oil,Wheat,Wheat flour,WSB,label
2255,0.9974,0.0000,0.2585,0.0000,0.0,0.7,0.0000,0.3078,0.0000,0.0000,...,0.0000,0.0000,0.1016,0.0000,0.0,0.1585,0.0000,0.0000,0.0000,0.1654
527,0.8218,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.2415,0.0000,0.1295,...,2.8683,0.0000,0.0000,0.0000,0.0,0.3599,0.0000,0.0000,0.7722,0.4390
58,0.7179,0.0000,0.0000,0.0000,0.0,0.7,0.1978,0.0779,0.5156,0.0000,...,0.0000,0.0000,0.0000,0.1047,0.0,0.1832,0.0000,0.0000,0.0000,0.2780
2509,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0160,0.3220,0.0715,0.6610,...,0.0000,0.0000,0.0000,0.0000,0.0,0.2795,4.1478,0.0000,0.5953,0.9462
3563,0.6752,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.3173,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0,0.2755,0.5276,3.6120,0.6411,0.9588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3582,0.6013,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.2900,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0,0.2478,0.7308,2.5267,0.6775,0.8347
3860,0.5178,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.2739,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0,0.2580,1.3447,2.5290,0.6889,0.8285
192,0.6190,0.4677,0.0000,0.0000,0.0,0.0,0.0000,0.0474,0.3906,0.0000,...,0.0000,0.1889,0.0000,0.0000,0.0,0.3578,0.0000,0.0000,0.7000,0.3939
321,0.5527,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.2881,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0,0.2615,0.9040,2.5152,0.7154,0.8015


# OptiCL: Optimization with Constraint Learning

## Step 1: Conceptual Model

In [3]:
def init_conceptual_model(cost):
    N = list(nutr_val.index)  # foods
    M = nutr_req.columns  # nutrient requirements

    model = ConcreteModel('TPDP')

    '''
    Decision variables
    '''
    model.x = Var(N, domain=NonNegativeReals)  # variables controlling the food basket

    '''
    Objective function.
    '''
    def obj_function(model):
        return sum(cost[food].item()*model.x[food] for food in N)

    model.OBJ = Objective(rule=obj_function, sense=minimize)

    '''
    Nutrients requirements constraint.
    '''
    def constraint_rule1(model, req):
        return sum(model.x[food] * nutr_val.loc[food, req] for food in N) >= nutr_req[req].item()
#     model.Constraint1 = Constraint(M, rule=constraint_rule1)
    
    return model

## Step 2: Data Processing

In [4]:
y = dataset['label']
X = dataset.drop(['label'], axis=1, inplace=False)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

## Part 3: Learn the predictive models
'alg_list' specifies the list of algorithms that you will consider in the training pipeline. If you have the InterpretableAI license, you can include **iai** (Optimal Trees with Hyperplanes) or **iai-single** (Optimal Trees with single feature splits) in the list. If using IAI, you must specify the metric as 'r2'. Otherwise, the default metric is 'neg_squared_mse'.

In [5]:
version = 'TR_test_v1'
alg_list = ['cart']
outcome_list = {'opticl_palatability': {'outcome_type': ['constraint', None], 'task_type': 'continuous', 'alg_list':alg_list, 
                                   'X_train':X_train, 'y_train':y_train, 'X_test':X_test, 'y_test':y_test}}

In [6]:
performance = pd.DataFrame()

if not os.path.exists('results/'):
    os.makedirs('results/')

for outcome in outcome_list.keys():
    print(f'Learning a constraint for {outcome}')
    
    alg_list = outcome_list[outcome]['alg_list']
    task_type = outcome_list[outcome]['task_type']
    for alg in alg_list:
        print(f'Training {alg}')
        X_train = outcome_list[outcome]['X_train']
        y_train = outcome_list[outcome]['y_train']
        X_test = outcome_list[outcome]['X_test']
        y_test = outcome_list[outcome]['y_test']
        
        if not os.path.exists('results/%s/' % alg):
            os.makedirs('results/%s/' % alg)
        s = 1

        ## Run shallow/small version of RF
        alg_run = 'rf_shallow' if alg == 'rf' else alg

        m, perf = opticl.run_model(X_train, y_train, X_test, y_test, alg_run, outcome, task = task_type,
                               seed = s, cv_folds = 5, 
                               # metric = 'r2',
                               save = False
                              )

        ## Save model
        constraintL = opticl.ConstraintLearning(X_train, y_train, m, alg)
        constraint_add = constraintL.constraint_extrapolation(task_type)
        constraint_add.to_csv('results/%s/%s_%s_model.csv' % (alg, version, outcome), index = False)

        ## Extract performance metrics
        try:
            perf['auc_train'] = roc_auc_score(y_train >= threshold, m.predict(X_train))
            perf['auc_test'] = roc_auc_score(y_test >= threshold, m.predict(X_test))
        except: 
            perf['auc_train'] = np.nan
            perf['auc_test'] = np.nan

        perf['seed'] = s
        perf['outcome'] = outcome
        perf['alg'] = alg
        perf['save_path'] = 'results/%s/%s_%s_model.csv' % (alg, version, outcome)
        
            
        perf.to_csv('results/%s/%s_%s_performance.csv' % (alg, version, outcome), index = False)
        
        performance = performance.append(perf)
        print()
print('Saving the performance...')
performance.to_csv('results/%s_performance.csv' % version, index = False)
print('Done!')

Learning a constraint for opticl_palatability
Training cart
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = cart, metric = None
saving... results/cart_opticl_palatability_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.012602897750181715
Train R2: 0.8198100498858901
-------------------testing evaluation-----------------------
Test MSE: 0.01905505909702495
Test R2: 0.7221888606223053

Saving the performance...
Done!


## Step 4: Predictive model selection and Optimization

In [7]:
version = 'TR_test_v1'
performance = pd.read_csv('results/%s_performance.csv' % version)
performance.dropna(axis='columns')

,save_path,seed,cv_folds,task,parameters,best_params,valid_score,train_score,train_r2,test_score,test_r2,outcome,alg
0,results/cart/TR_test_v1_opticl_palatability_mo...,1,5,continuous,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 10, 'max_features': 1.0, 'min_sa...",-0.019925,0.012603,0.81981,0.019055,0.722189,opticl_palatability,cart


In [8]:
model_master = opticl.model_selection(performance, outcome_list)
model_master

               outcome model_type  \
0  opticl_palatability       cart   

                                           save_path        task  objective  
0  results/cart/TR_test_v1_opticl_palatability_mo...  continuous          0  


,outcome,model_type,save_path,task,objective
0,opticl_palatability,cart,results/cart/TR_test_v1_opticl_palatability_mo...,continuous,0


In [9]:
def check_palatability(solution): 
    min_, max_ = 72.401310506, 430.8811249455  #### based on the dataset
    group = {0: [1, 10, 13, 11, 14, 20, 21], 1: [0, 6, 9, 12], 2: [19], 3: [5, 22, 15, 16, 17, 18], 4: [2, 3, 4, 7, 8]}
    coeff = [1, 5.7, 16, 4.4, 6.6]
    opt = [400, 130/2, 55/2, 45, 30]
    
    sol_score = np.round(np.sqrt(sum((coeff[i]*(sum(solution[j] for j in group[i])-opt[i]))**2 for i in range(5))), 10)
    norm_score = 1-(sol_score - min_)/(max_ - min_)
    return norm_score

In [10]:
model_master = opticl.model_selection(performance, outcome_list)
model_master[['lb', 'ub', 'SCM_counterfactuals', 'features', 'trust_region', 'dataset_path',
              'clustering_model', 'max_violation', 'enlargement', 'var_features', 'contex_features']] = None

model_master.loc[0, 'lb'] = 0.5
model_master.loc[0, 'ub'] = None
model_master.loc[0, 'SCM_counterfactuals'] = None
model_master.at[0, 'features'] = [col for col in X.columns]
model_master.loc[0, 'trust_region'] = True
model_master.loc[0, 'dataset_path'] = '../WFP/processed-data/WFP_dataset_processed.csv'
model_master.loc[0, 'clustering_model'] = None
model_master.loc[0, 'max_violation'] = None
model_master.at[0, 'var_features'] = [col for col in X.columns]
model_master.at[0, 'contex_features'] = {}  # example: {'contextual_feat_name_1': 1, contextual_feat_name_2': 5}

# enlargement option: 0-No enlargement, 1-CH enlargement; 
# enlargement constraint:  0-constraint, 1-objective penalty; 
# constraint ub/penalty multiplier
model_master.at[0, 'enlargement'] = [1, 0, 0] 
model_master

               outcome model_type  \
0  opticl_palatability       cart   

                                           save_path        task  objective  
0  results/cart/TR_test_v1_opticl_palatability_mo...  continuous          0  


,outcome,model_type,save_path,task,objective,lb,ub,SCM_counterfactuals,features,trust_region,dataset_path,clustering_model,max_violation,enlargement,var_features,contex_features
0,opticl_palatability,cart,results/cart/TR_test_v1_opticl_palatability_mo...,continuous,0,0.5,None,None,"[Beans, Bulgur, Cheese, Fish, Meat, CSB, Dates...",True,../WFP/processed-data/WFP_dataset_processed.csv,None,None,"[1, 0, 0]","[Beans, Bulgur, Cheese, Fish, Meat, CSB, Dates...",{}


In [ ]:
reload(em)
solution_info = {}

# epsilon = [0.00, 0.010, 0.050, 0.075] + list(np.round(np.logspace(-1,-0.096910013,15, base=10), 2))
epsilon = list(np.round(np.linspace(0, 0.49, 25), 3)) + list(np.round(np.linspace(0.5, 0.79, 15), 3))
N = len(epsilon)
M = 200
E0 = 0
E1 = .8
# epsilon = np.round(np.linspace(E0,E1,N), 4)

for i, eps in enumerate(epsilon):
    for j in range(M):
        np.random.seed(j)  #########################
        model_master.at[0, 'enlargement'] = [1, 0, eps] 
        cost = pd.DataFrame([np.round(np.random.rand(23), 4)], columns=X.columns)
        conceptual_model = init_conceptual_model(cost)
        final_model = em.optimization_MIP(conceptual_model, model_master)
        opt = SolverFactory('gurobi')
        print('---------------------Solving the optimization problem---------------------')
        results = opt.solve(final_model) 
        solution = []
        for food in list(nutr_val.index):
            solution.append(np.round(value(final_model.x[food])*100, 6))
        solution_info[i,j] = {'solution': solution, 'obj': value(final_model.OBJ), 'real_pal': check_palatability(solution), 'pred_pal': value(final_model.y['opticl_palatability'])}
#         print('###################################################################################')
#         print('avg real palatability', [np.mean([solution_info[ii,jj]['real_pal'] for jj in range(j)]) for ii in range(i)])
#         print('avg objective value', [np.mean([solution_info[ii,jj]['obj'] for jj in range(j)]) for ii in range(i)])
#         print('avg pred palatability', [np.mean([solution_info[ii,jj]['pred_pal'] for jj in range(j)]) for ii in range(i)])
#         print('###################################################################################')

Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem--

The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlar

Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem--

The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlar

Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.0.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem--

...

Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.5.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.5.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.5.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem--

The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the

The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the

The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the

The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the enlarged CH trust region
The trust region is being enlarged with a constraint upper bounded by: 0.521.
... Trust region defined.
Embedding constraints for opticl_palatability
---------------------Solving the optimization problem---------------------
Generating constraints for the trust region using 4049 samples.
The l1 norm is used for the

In [ ]:
lb = model_master.loc[0, 'lb']
import pickle
filename = f'LB_{lb}_{alg}_eps{E0}-{E1}_N{N}_M{M}.pkl'
with open(filename, 'wb') as f:
    pickle.dump(solution_info, f)

### Plot mean objective value and mean real palatability wrt epsilon

In [ ]:
mean_obj = [np.median([solution_info[i,j]['obj'] for j in range(M)]) for i in range(N)]
improvement_obj = [(np.mean([solution_info[0,j]['obj'] for j in range(M)]) - np.mean([solution_info[i,j]['obj'] for j in range(M)]))/np.mean([solution_info[0,j]['obj'] for j in range(M)]) for i in range(N)]
mean_real_pal = [np.median([solution_info[i,j]['real_pal'] for j in range(M)]) for i in range(N)]
violations_pal = [np.sum([1 if solution_info[i,j]['real_pal'] < lb else 0 for j in range(M)])/M  for i in range(N)]

# plt.plot(epsilon, improvement_obj, color='black', label='avg % of improv in the obj', linestyle='dotted')
plt.plot(epsilon, mean_obj, color='black', label='avg obj')
plt.plot(epsilon, mean_real_pal, color='gray', label='avg real palat')
plt.plot(epsilon, [lb]*len(epsilon), color='black', label='palat threshold', linestyle='--')
# plt.plot(epsilon, violations_pal, color='gray', label='avg % of violations', linestyle='-.')
plt.legend()
# plt.gca().xaxis.grid(True, which='both', linewidth=0.5)
plt.axvspan(0, 0.28, color='k', alpha=0.2)
# plt.ylabel('average')
plt.xlabel('epsilon')
plt.ylim(0,1.1)
plt.title(f'$\epsilon$-CH with {alg}')
plt.show()

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d

mean_obj_smoothed = gaussian_filter1d(mean_obj, sigma=5)
mean_real_pal_smoothed = gaussian_filter1d(mean_real_pal, sigma=1)

plt.plot(epsilon, mean_obj_smoothed, color='black', label='avg obj')
plt.plot(epsilon, mean_real_pal_smoothed, color='lightgray', label='avg real palat')
plt.plot(epsilon, [lb]*len(epsilon), color='black', label='palat threshold', linestyle='--')
# plt.plot(epsilon, violations_pal, color='lightgray', label='% of violations', linestyle='-.')
plt.legend()
plt.gca().xaxis.grid(True, which='both', linewidth=0.5)
# plt.ylabel('average')
plt.xlabel('epsilon')
plt.title('$\epsilon$-CH with CART smoothed')
plt.show()

In [ ]:
mean_obj_smoothed = gaussian_filter1d(mean_obj, sigma=3)
mean_real_pal_smoothed = gaussian_filter1d(mean_real_pal, sigma=3)

plt.plot(mean_obj_smoothed, mean_real_pal_smoothed, color='black')
plt.xlabel('mean obj')
plt.ylabel('mean real palat')
plt.show()

### Plot mean objective value and number of palatability constr violations wrt epsilon

In [ ]:
mean_obj = [np.mean([solution_info[i,j]['obj'] for j in range(M)]) for i in range(N)]
violations_pal = [np.sum([1 if solution_info[i,j]['real_pal'] < lb else 0 for j in range(M)])/M  for i in range(N)]

plt.plot(epsilon, mean_obj, color='black', label='avg obj')
plt.plot(epsilon, violations_pal, color='lightgray', label='% of violations')
plt.legend()
plt.gca().xaxis.grid(True, which='both', linewidth=0.5)
# plt.ylabel('average')
plt.xlabel('epsilon')
plt.show()

In [ ]:
mean_obj = [np.mean([solution_info[i,j]['obj'] for j in range(M)]) for i in range(N)]
violations_pal = [np.sum([1 if solution_info[i,j]['real_pal'] < lb else 0 for j in range(M)])/M  for i in range(N)]

plt.scatter(mean_obj, violations_pal, color='gray', edgecolors='k')
plt.gca().xaxis.grid(True, which='both', linewidth=0.5)
plt.show()

In [ ]:
med_obj, med_pal = [np.median([solution_info[i,j]['obj'] for j in range(M)]) for i in range(N)],[np.median([solution_info[i,j]['real_pal'] for j in range(M)]) for i in range(N)]
plt.scatter(-np.array(med_pal), -np.array(med_obj), color='gray', edgecolors='k')
plt.show()